# Explore here

Se han recopilado datos sociodemográficos y de recursos de salud por condado en los Estados Unidos y queremos descubrir si existe alguna relación entre los recursos sanitarios y los datos sociodemográficos.

In [30]:
# Your code here
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge


In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/regularized-linear-regression-project-tutorial/main/demographic_health_data.csv")


In [4]:

pd.set_option('display.max_rows',110)
pd.set_option('display.max_columns',110)  # Max number of columns to display
data.describe().T

,count,mean,std,min,25%,50%,75%,max
fips,3140.0,30401.640764,15150.559265,1001.000000,18180.500000,29178.000000,45081.500000,5.604500e+04
TOT_POP,3140.0,104189.412420,333583.395432,88.000000,10963.250000,25800.500000,67913.000000,1.010552e+07
0-9,3140.0,12740.302866,41807.301846,0.000000,1280.500000,3057.000000,8097.000000,1.208253e+06
0-9 y/o % of total pop,3140.0,11.871051,2.124081,0.000000,10.594639,11.802727,12.951840,2.546068e+01
19-Oct,3140.0,13367.976752,42284.392134,0.000000,1374.500000,3274.000000,8822.250000,1.239139e+06
10-19 y/o % of total pop,3140.0,12.694609,1.815044,0.000000,11.674504,12.687422,13.659282,2.330437e+01
20-29,3140.0,14469.331529,49577.725396,0.000000,1263.750000,3108.000000,8976.250000,1.557073e+06
20-29 y/o % of total pop,3140.0,12.283979,3.126297,0.000000,10.496774,11.772649,13.182260,3.757020e+01
30-39,3140.0,13916.490127,48990.951710,11.000000,1232.750000,3000.500000,8314.250000,1.501844e+06
30-39 y/o % of total pop,3140.0,11.751535,1.696599,6.092789,10.689322,11.580861,12.639379,2.222513e+01


In [5]:
data.shape

(3140, 108)

In [6]:
if data.duplicated:
    data.drop_duplicates

data.head()

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,40-49,40-49 y/o % of total pop,50-59,50-59 y/o % of total pop,60-69,60-69 y/o % of total pop,70-79,70-79 y/o % of total pop,80+,80+ y/o % of total pop,White-alone pop,% White-alone,Black-alone pop,% Black-alone,Native American/American Indian-alone pop,% NA/AI-alone,Asian-alone pop,% Asian-alone,Hawaiian/Pacific Islander-alone pop,% Hawaiian/PI-alone,Two or more races pop,% Two or more races,POP_ESTIMATE_2018,N_POP_CHG_2018,GQ_ESTIMATES_2018,R_birth_2018,R_death_2018,R_NATURAL_INC_2018,R_INTERNATIONAL_MIG_2018,R_DOMESTIC_MIG_2018,R_NET_MIG_2018,Less than a high school diploma 2014-18,High school diploma only 2014-18,Some college or associate's degree 2014-18,Bachelor's degree or higher 2014-18,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a high school diploma only 2014-18,Percent of adults completing some college or associate's degree 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,POVALL_2018,PCTPOVALL_2018,PCTPOV017_2018,PCTPOV517_2018,MEDHHINC_2018,CI90LBINC_2018,CI90UBINC_2018,Civilian_labor_force_2018,Employed_2018,Unemployed_2018,Unemployment_rate_2018,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018,Active Physicians per 100000 Population 2018 (AAMC),Total Active Patient Care Physicians per 100000 Population 2018 (AAMC),Active Primary Care Physicians per 100000 Population 2018 (AAMC),Active Patient Care Primary Care Physicians per 100000 Population 2018 (AAMC),Active General Surgeons per 100000 Population 2018 (AAMC),Active Patient Care General Surgeons per 100000 Population 2018 (AAMC),Total nurse practitioners (2019),Total physician assistants (2019),Total Hospitals (2019),Internal Medicine Primary Care (2019),Family Medicine/General Practice Primary Care (2019),Total Specialist Physicians (2019),ICU Beds_x,Total Population,Population Aged 60+,Percent of Population Aged 60+,COUNTY_NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,county_pop2018_18 and older,anycondition_prevalence,anycondition_Lower 95% CI,anycondition_Upper 95% CI,anycondition_number,Obesity_prevalence,Obesity_Lower 95% CI,Obesity_Upper 95% CI,Obesity_number,Heart disease_prevalence,Heart disease_Lower 95% CI,Heart disease_Upper 95% CI,Heart disease_number,COPD_prevalence,COPD_Lower 95% CI,COPD_Upper 95% CI,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
0,1001,55601,6787,12.206615,7637,13.735364,6878,12.370281,7089,12.749771,7582,13.636445,7738,13.917016,5826,10.478229,4050,7.284042,2014,3.622237,42660,76.725239,10915,19.630942,267,0.480207,681,1.224798,62,0.111509,1016,1.827305,55601,158,455,11.8,9.6,2.2,0.0,0.7,0.6,4204,12119,10552,10291,11.3,32.6,28.4,27.7,7587,13.8,19.3,19.5,59338,53628,65048,25957,25015,942,3.6,59338,119.0,217.1,196.7,77.2,71.2,7.6,6.9,28.859137,6.085786,1.148905,25.992561,21.249061,72.142154,6,55036,10523,19.1,Autauga,Alabama,1,1,42438,47.6,45.4,49.4,20181,35.8,34.2,37.3,15193,7.9,7.2,8.7,3345,8.6,7.3,9.9,3644,12.9,11.9,13.8,5462,3.1,2.9,3.3,1326,3
1,1003,218022,24757,11.355276,26913,12.344167,23579,10.814964,25213,11.564429,27338,12.539102,29986,13.753658,29932,13.728890,20936,9.602701,9368,4.296814,190301,87.285228,19492,8.940382,1684,0.772399,2508,1.150343,146,0.066966,3891,1.784682,218022,5403,2190,10.5,10.3,0.1,0.5,24.3,24.8,14310,40579,46025,46075,9.7,27.6,31.3,31.3,21069,9.8,13.9,13.1,57588,54437,60739,93849,90456,3393,3.6,57588,115.5,217.1,196.7,77.2,71.2,7.6,6.9,113.162114,23.863512,4.505074,101.921730,83.321572,282.882982,51,203360,53519,26.3,Baldwin,Alabama,1,3,170912,40.2,38.2,42.3,68790,29.7,28.4,31.0,50761,7.8,7.0,8.7,13414,8.6,7.2,10.1,14692,12.0,11.0,13.1,20520,3.2,3.0,3.5,5479,4
2,1005,24881,2732,10.980266,2960,11.896628,3268,13.134520,3201,12.865239,3074,12.354809,3278,13.174712,3076,12.3

In [7]:
data.isna().sum().sum()

0

In [8]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Columns: 108 entries, fips to Urban_rural_code
dtypes: float64(61), int64(45), object(2)
memory usage: 2.6+ MB
None


In [9]:
cat_data = data.select_dtypes(include=['object']).columns

****Selección de Características****

Tendremos que seleccionar las mejores características para pasarle al modelo.


Busqueda de correlaciones: 

Variable Objetivo = Heart Disease Number

In [10]:
# Correlaciones


num_data = data.select_dtypes(exclude=['object'])
corr_data = num_data.corr()

np_corr = np.where(corr_data >= 0.15, corr_data, 0)
corr = pd.DataFrame(np_corr, columns=corr_data.columns, index=corr_data.index)

hd_np_corr = corr['Heart disease_number']

corr['Heart disease_number'][corr['Heart disease_number'] > 0]


print(f"Hay: {len(hd_np_corr[(hd_np_corr > 0)])} variables predictoras que presentan una alta correlación con la variable objetivo")


Hay: 57 variables predictoras que presentan una alta correlación con la variable objetivo


In [11]:
hd_np_corr[(hd_np_corr > 0) & (hd_np_corr < 0.9)]
                    

20-29 y/o % of total pop                                                  0.150898
30-39 y/o % of total pop                                                  0.272543
40-49 y/o % of total pop                                                  0.175525
Black-alone pop                                                           0.814667
Native American/American Indian-alone pop                                 0.729656
Asian-alone pop                                                           0.784295
% Asian-alone                                                             0.440553
Hawaiian/Pacific Islander-alone pop                                       0.450083
Two or more races pop                                                     0.869900
N_POP_CHG_2018                                                            0.453290
GQ_ESTIMATES_2018                                                         0.876647
R_NATURAL_INC_2018                                                        0.154453
R_IN

In [12]:
# Escalado de Datos

sc_features = num_data.drop('Heart disease_number', axis=1)

scaler= StandardScaler()
sc_data = scaler.fit_transform(sc_features)

sc_data = pd.DataFrame(sc_data, columns=sc_features.columns)
sc_data['Heart disease_number'] = num_data['Heart disease_number']


In [13]:
# Train - Test Split

x = sc_data.drop('Heart disease_number', axis=1)
y= sc_data['Heart disease_number']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)


In [14]:
# Crear un modelo XGBoost Regressor

model = xgb.XGBRegressor(objective='reg:squarederror')
model.fit(X_train, y_train)
feature_importances = model.feature_importances_

feature_names = x.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_feature_importance = dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True))

# Imprime la importancia de las características (número adimensional, las características se comparan entre sí en términos de su contribución relativa a la predicción)
for feature, importance in sorted_feature_importance.items():
    print(f'Feature: {feature}, Importance: {importance*100}')



Feature: Some college or associate's degree 2014-18, Importance: 44.676610827445984
Feature: CKD_number, Importance: 16.506323218345642
Feature: ICU Beds_x, Importance: 8.564156293869019
Feature: COPD_number, Importance: 8.547797054052353
Feature: 80+, Importance: 6.079520285129547
Feature: Unemployed_2018, Importance: 5.508676543831825
Feature: Native American/American Indian-alone pop, Importance: 3.944692760705948
Feature: GQ_ESTIMATES_2018, Importance: 1.699117198586464
Feature: Population Aged 60+, Importance: 1.2176980264484882
Feature: 70-79, Importance: 1.1365041136741638
Feature: Less than a high school diploma 2014-18, Importance: 0.7408201694488525
Feature: anycondition_number, Importance: 0.427310261875391
Feature: 30-39, Importance: 0.4102275241166353
Feature: High school diploma only 2014-18, Importance: 0.2521711867302656
Feature: Med_HH_Income_Percent_of_State_Total_2018, Importance: 0.0962704187259078
Feature: diabetes_number, Importance: 0.026878900825977325
Feature: 

In [15]:
def get_vars_to_delete(d_imp, corr, imp_cut = 0.1, corr_cut = 0.15):
  ls_vars_low_corr = corr[corr['Heart disease_number'].abs()<=corr_cut].index
  return [var for var in ls_vars_low_corr if d_imp[var]<=imp_cut]


vars_to_drop = get_vars_to_delete(sorted_feature_importance, corr_data, imp_cut=0.1, corr_cut=0.15)
vars_to_drop

['fips',
 '0-9 y/o % of total pop',
 '10-19 y/o % of total pop',
 '50-59 y/o % of total pop',
 '% Black-alone',
 '% NA/AI-alone',
 '% Hawaiian/PI-alone',
 '% Two or more races',
 'R_birth_2018',
 'R_DOMESTIC_MIG_2018',
 'R_NET_MIG_2018',
 'Percent of adults with less than a high school diploma 2014-18',
 "Percent of adults completing some college or associate's degree 2014-18",
 'PCTPOVALL_2018',
 'PCTPOV017_2018',
 'PCTPOV517_2018',
 'Unemployment_rate_2018',
 'Active Primary Care Physicians per 100000 Population 2018 (AAMC)',
 'Active Patient Care Primary Care Physicians per 100000 Population 2018 (AAMC)',
 'Active General Surgeons per 100000 Population 2018 (AAMC)',
 'Active Patient Care General Surgeons per 100000 Population 2018 (AAMC)',
 'STATE_FIPS',
 'CNTY_FIPS',
 'diabetes_Lower 95% CI']

In [16]:
X_train.drop(vars_to_drop, axis = 1, inplace=True)
X_test.drop(vars_to_drop, axis = 1, inplace=True)

In [17]:
var_dup = ["20-29", "30-39", "40-49", "50-59", "50-59", "70-79", "80+" , "White-alone pop", "Asian-alone pop" ]
X_train.drop(var_dup, axis = 1, inplace=True)
X_test.drop(var_dup, axis = 1, inplace=True)

***MODELIZADO CON LINEAR

Ya hecha la selección de características ahora voy a instanciar y modelizar los datos. 

In [26]:
model = LinearRegression()

resultado = model.fit(X_train, y_train)

In [27]:
#Intercepto del Modelo

model.intercept_

5596.386681064318

In [28]:
# El coeficiente de las Variables Predictoras

coef = dict(zip(X_train.columns, model.coef_)) 
dict(sorted(coef.items(), key=lambda x: x[1], reverse=True))

{'TOT_POP': 1115715970229852.8,
 'CKD_number': 7951.5234375,
 'COPD_number': 6632.47265625,
 'Population Aged 60+': 4265.30078125,
 'diabetes_number': 3743.671875,
 'Obesity_number': 2625.34375,
 "Bachelor's degree or higher 2014-18": 2571.4375,
 'High school diploma only 2014-18': 1756.875,
 'anycondition_prevalence': 1464.80859375,
 '0-9': 1231.34375,
 'Heart disease_Upper 95% CI': 1044.25,
 'Obesity_Upper 95% CI': 809.625,
 'diabetes_prevalence': 503.09375,
 'POVALL_2018': 473.203125,
 'Total Hospitals (2019)': 443.6875,
 'Total physician assistants (2019)': 415.5,
 'Heart disease_Lower 95% CI': 372.015625,
 "Some college or associate's degree 2014-18": 344.1640625,
 'GQ_ESTIMATES_2018': 219.1328125,
 'Employed_2018': 181.1796875,
 'CI90LBINC_2018': 131.96875,
 'Civilian_labor_force_2018': 114.015625,
 'Two or more races pop': 110.1328125,
 'CKD_Lower 95% CI': 93.359375,
 'Obesity_Lower 95% CI': 59.09375,
 'Active Physicians per 100000 Population 2018 (AAMC)': 42.03125,
 '% Asian-al

In [21]:
y_pred = model.predict(X_test)

In [23]:
print(f"MSE: {mean_squared_error(y_test, y_pred)}")
print(f"R2 Score: {r2_score(y_test, y_pred)}")


MSE: 181276.42345283268
R2 Score: 0.9984864926342157


**OPTIMIZACIÓN**

In [33]:
lasso = Lasso(alpha=0.1, max_iter=300)
lasso.fit(X_train, y_train)

y_hat = lasso.predict(X_test)

score = lasso.score(X_test, y_test)
print("R2 score:", score)

R2 score: 0.9950475638315365


/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.160e+08, tolerance: 6.713e+07
  model = cd_fast.enet_coordinate_descent(


In [34]:
print(f"MSE: {mean_squared_error(y_hat, y_pred)}")
print(f"R2 Score: {r2_score(y_hat, y_pred)}")

MSE: 358455.77037922887
R2 Score: 0.9971190351991996
